In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import sqldf as sd

In [2]:
results = pd.read_csv('datasets/cleaned/race_results.csv')
races = pd.read_csv('datasets/cleaned/races.csv')
drivers = pd.read_csv('datasets/cleaned/drivers.csv')

In [3]:
master_stats = pd.merge(results, drivers, on='driverId', how='left')
master_stats = pd.merge(master_stats, races, on='raceId', how='left').rename(columns={'name_x': 'driverName', 'name_y': 'raceName'})
master_stats = master_stats.query('year != 2022')
master_stats.head()

,raceId,driverId,constructorId,grid,positionOrder,points,laps,time,milliseconds,fastestLap,...,statusId,driverRef,dob,nationality,driverName,year,round,circuitId,raceName,date
0,18,1,1,1,1,10.0,58,0 days 01:34:50.616000,5690616.0,39.0,...,1,hamilton,1985-01-07,British,Lewis Hamilton,2008,1,1,Australian Grand Prix,2008-03-16
1,18,2,2,5,2,8.0,58,0 days 01:34:56.094000,5696094.0,41.0,...,1,heidfeld,1977-05-10,German,Nick Heidfeld,2008,1,1,Australian Grand Prix,2008-03-16
2,18,3,3,7,3,6.0,58,0 days 01:34:58.779000,5698779.0,41.0,...,1,rosberg,1985-06-27,German,Nico Rosberg,2008,1,1,Australian Grand Prix,2008-03-16
3,18,4,4,11,4,5.0,58,0 days 01:35:07.797000,5707797.0,58.0,...,1,alonso,1981-07-29,Spanish,Fernando Alonso,2008,1,1,Australian Grand Prix,2008-03-16
4,18,5,1,3,5,4.0,58,0 days 01:35:08.630000,5708630.0,43.0,...,1,kovalainen,1981-10-19,Finnish,Heikki Kovalainen,2008,1,1,Australian Grand Prix,2008-03-16


In [4]:
all_points = master_stats.groupby('driverName').agg({'points': 'sum'}).sort_values(by='points',ascending=False).reset_index()
all_points.head(10)

,driverName,points
0,Lewis Hamilton,4163.5
1,Sebastian Vettel,3061.0
2,Fernando Alonso,1980.0
3,Kimi Räikkönen,1873.0
4,Valtteri Bottas,1731.0
5,Nico Rosberg,1594.5
6,Michael Schumacher,1566.0
7,Max Verstappen,1550.5
8,Daniel Ricciardo,1273.0
9,Jenson Button,1235.0


In [5]:
all_wins = master_stats.query('positionOrder == 1').groupby('driverName').agg({'positionOrder': 'count'}).sort_values(
    by='positionOrder', ascending=False).rename(columns={'positionOrder': 'careerWins'}).reset_index()
all_wins.head(10)

,driverName,careerWins
0,Lewis Hamilton,103
1,Michael Schumacher,91
2,Sebastian Vettel,53
3,Alain Prost,51
4,Ayrton Senna,41
5,Fernando Alonso,32
6,Nigel Mansell,31
7,Jackie Stewart,27
8,Jim Clark,25
9,Niki Lauda,25


In [6]:
all_podiums = master_stats[master_stats['positionOrder'].isin([1,2,3])].groupby('driverName').agg({'positionOrder': 'count'}).sort_values(
    by='positionOrder', ascending=False).rename(columns={'positionOrder': 'podiums'}).reset_index()
all_podiums.head(10)


,driverName,podiums
0,Lewis Hamilton,182
1,Michael Schumacher,155
2,Sebastian Vettel,122
3,Alain Prost,106
4,Kimi Räikkönen,103
5,Fernando Alonso,98
6,Ayrton Senna,80
7,Rubens Barrichello,68
8,Valtteri Bottas,67
9,David Coulthard,62


In [7]:
all_poles = master_stats.query('grid == 1').groupby('driverName').agg({'positionOrder': 'count'}).sort_values(
    by='positionOrder', ascending=False).rename(columns={'positionOrder': 'poles'}).reset_index()
all_poles.head(10)


,driverName,poles
0,Lewis Hamilton,103
1,Michael Schumacher,68
2,Ayrton Senna,65
3,Sebastian Vettel,57
4,Jim Clark,34
5,Alain Prost,33
6,Nigel Mansell,32
7,Nico Rosberg,30
8,Juan Fangio,29
9,Mika Häkkinen,26


In [8]:
all_champs = master_stats.groupby(['year', 'driverName']).agg({'points': 'sum'}).reset_index()
all_champs = all_champs.sort_values(by='points', ascending=False).groupby('year').first().reset_index()

all_champs = all_champs.groupby('driverName').agg({'driverName': 'count'}).rename(columns={
    'driverName': 'championships'}).sort_values(by='championships', ascending=False).reset_index()

# Due to some very weird rules back in the day the 1988 season was won by Senna instead of Prost
all_champs.loc[all_champs['driverName'] == 'Alain Prost', 'championships'] = 4
all_champs.loc[all_champs['driverName'] == 'Ayrton Senna', 'championships'] = 3
all_champs = all_champs.sort_values(by='championships', ascending=False)
all_champs.head(10)

,driverName,championships
0,Michael Schumacher,7
1,Lewis Hamilton,7
3,Juan Fangio,5
2,Alain Prost,4
4,Sebastian Vettel,4
5,Niki Lauda,3
6,Graham Hill,3
7,Jack Brabham,3
8,Jackie Stewart,3
9,Nelson Piquet,3


In [9]:
highest_wins = master_stats.query('positionOrder == 1').groupby(['driverName','year']).agg(
    {'positionOrder': 'count'}).rename(columns={'positionOrder': 'highestWins'}).reset_index()
highest_wins = highest_wins.sort_values(by='highestWins', ascending=False).groupby('driverName').first().reset_index()
highest_wins.head(10)

,driverName,year,highestWins
0,Alain Prost,1993,7
1,Alan Jones,1980,5
2,Alberto Ascari,1952,6
3,Alessandro Nannini,1989,1
4,Ayrton Senna,1988,8
5,Bill Vukovich,1954,1
6,Bob Sweikert,1955,1
7,Bruce McLaren,1960,1
8,Carlos Pace,1975,1
9,Carlos Reutemann,1978,4


In [10]:
races_in_year = master_stats.groupby('year').agg({'raceId':'nunique'}).rename(columns={'raceId': 'races'}).reset_index()
highest_wins = pd.merge(highest_wins, races_in_year, on='year', how='inner')
highest_wins['highestWinPercentage'] = highest_wins['highestWins'] / highest_wins['races'] * 100
highest_wins

,driverName,year,highestWins,races,highestWinPercentage
0,Alain Prost,1993,7,16,43.750000
1,Alan Jones,1980,5,14,35.714286
2,Jean-Pierre Jabouille,1980,1,14,7.142857
3,Alberto Ascari,1952,6,8,75.000000
4,Piero Taruffi,1952,1,8,12.500000
...,...,...,...,...,...
106,Riccardo Patrese,1991,2,16,12.500000
107,Richie Ginther,1965,1,10,10.000000
108,Rubens Barrichello,2002,4,17,23.529412
109,Sam Hanks,1957,1,8,12.500000


In [11]:
stats_df = pd.merge(all_wins, all_champs, on='driverName', how='left')
stats_df = pd.merge(stats_df, all_podiums, on='driverName', how='inner')
stats_df = pd.merge(stats_df, all_poles, on='driverName', how='inner')
stats_df = pd.merge(stats_df, all_points, on='driverName', how='inner')
stats_df = pd.merge(stats_df, highest_wins, on='driverName', how='inner')
stats_df.drop(columns={'year', 'races'}, inplace=True)
stats_df.head(16)

,driverName,careerWins,championships,podiums,poles,points,highestWins,highestWinPercentage
0,Lewis Hamilton,103,7.0,182,103,4163.50,11,64.705882
1,Michael Schumacher,91,7.0,155,68,1566.00,13,72.222222
2,Sebastian Vettel,53,4.0,122,57,3061.00,13,68.421053
3,Alain Prost,51,4.0,106,33,798.50,7,43.750000
4,Ayrton Senna,41,3.0,80,65,614.00,8,50.000000
5,Fernando Alonso,32,2.0,98,22,1980.00,7,38.888889
6,Nigel Mansell,31,1.0,59,32,482.00,9,56.250000
7,Jackie Stewart,27,3.0,43,17,360.00,6,54.545455
8,Jim Clark,25,2.0,32,34,274.00,7,70.000000
9,Niki Lauda,25,3.0,54,24,420.50,5,31.250000
